# e1_extract_occ_gen
From the file list_meetings.csv, find the number of invertions of each entity for one issue. 

In [1]:
import urllib.request
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tokenize import MWETokenizer
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import csv


importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb


In [7]:
def write_occurrence_issue(occurences_meetings, s):
        with open(s, "w", newline='') as file:
                writer = csv.writer(file)
        #header
                writer.writerow(('entity','interventions'))
                writer.writerows(occurences_meetings)

In [8]:
def open_list_meetings():
    """ Open the csv file that contain all the meetings. """
    f = open('list_meetings.csv')
    return csv.reader(f)

In [9]:
def extract_tuple(line):
    """ Extract tuple for each row from Paula's dataset. """
    l = line.replace('"',"")
    l = l.replace('\n',"")
    l = l.split('\t')
    return l

In [10]:
def clean_tp(sentence):
    """ Clean the sentence by removing special char. """
    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)


In [11]:
def count_occurences(list_sentences, dict_occ, tokenizer1, tokenizer2, tokenizer3, list_entities, number):
    """ Count number of time each entity in list_entities is mentioned in list_sentences. """

    for s in list_sentences:
        #Split line into words with tokenizer to detetc entity
        line = s.replace(",","")
        line_splited = word_tokenize(line)
        tokens = tokenizer1.tokenize(line_splited) 
        tokens = tokenizer2.tokenize(tokens) 
        tokens = tokenizer3.tokenize(tokens) 
        tokens = [clean_tp(token) for token in tokens]
        tokens_c = []
        for i in range(len(tokens)-1):
            if(tokens[i+1] !='.'):
                tokens_c.append(tokens[i])

        for entity in list_entities:
            #Increment value of intervention of the entity
            if(entity in tokens_c):
                dict_occ[entity] += tokens_c.count(entity)               
    rows = [(number, entity, dict_occ[entity]) for entity in dict_occ]
    return rows

In [15]:
def extract_occurences_issue_ENB(list_sentences, number):
    """ Extract all the occurences for each entities for a specific issue. """
    #List sentences
    sentences = list_tp = list_sentences

    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_clean.txt'))]
    list_entities = [s.replace(',','') for s in list_entities]
    list_entities = [s.replace(':','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]

    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')

    occurences_meetings = []

    dict_occurences = dict.fromkeys(list_entities, 0)
    occurences_meetings = count_occurences(sentences, dict_occurences, tokenizer1,tokenizer2,tokenizer3, list_entities, number)
    return occurences_meetings

In [16]:
def extract_from_csv_list_issues(csv_file):
    """ Extract from "csv_file" all the html link to be able to extract all the <p> tags. """
    f = open(csv_file)
    csv_f = csv.reader(f)
    list_pt = []
    l =list(csv_f)[1:]
    issue = []
    for x in l:
        issue.append(int(x[4]))
    return issue


import csv
issues = extract_from_csv_list_issues('Text/list_meetings.csv')
occ = []
for x in issues:
    p = c1.extract_paragraphes_from_issue(x)
    s = c2.extract_from_txt_sentences(p)
    e = extract_occurences_issue_ENB(s, x)
    occ_i = np.sum([x[2] for x in e])
    occ.append(occ_i)

sums = np.sum(occ)

import collections
from scipy.stats import pareto
import matplotlib.pyplot as plt 
def node_degree_distribution(occ):
    degree_sequence = sorted(occ, reverse=True)
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    """Plot the distribution of node degrees"""
    figsize = (20,10)
    fig, axe = plt.subplots(1, 1, figsize=figsize)


    csfont = {'fontname':'Helvetica','fontsize':'18' }
    hfont = {'fontname':'Helvetica','fontsize':'14'}
    plt.title('Distribution of the number of intervention per documents',**csfont)
    plt.xlabel('Number of interventions', **hfont)
    plt.ylabel('Number of of documents', **hfont)
    plt.plot(deg,cnt)

    plt.show()

node_degree_distribution(occ)